In [48]:
%pip install groq

In [49]:
import os
os.environ['GROQ_API_KEY']= "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"

In [50]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's technology landscape, and their importance can be understood from several perspectives:

1. **Efficient Processing**: Fast language models enable efficient processing of vast amounts of text data, which is essential in various applications such as:
	* Sentiment analysis
	* Text classification
	* Language translation
	* Summarization
	* Question answering
	* Chatbots and conversational AI

By quickly processing text data, fast language models facilitate real-time interactions, reducing latency and improving overall user experience.

2. **Scalability**: Fast language models are scalable, allowing them to handle large volumes of text data without compromising performance. This is particularly important in applications such as:
	* Social media monitoring
	* Customer service chatbots
	* Content moderation
	* Document analysis

Scalability ensures that language models can handle increasing amounts of data without becoming bottlenecked, enabling bus

In [51]:
class Agent:
  def __init__(self, client, system):
    self.client = client
    self.system = system #System Prompt, you work in a loop of action, thought, and observation
    self.messages = []
    if self.system is not None: #Initial System messages to the Language Model
      self.messages.append({"role": "system", "content": self.system})

  def __call__(self, message=""):
    if message:
      self.messages.append({"role" : "user", "content" : message })
    result = self.execute()
    self.messages.append({"role" : "assistant", "content": result})
    return result

  def execute(self):
    completion = client.chat.completions.create(
        messages=self.messages,
        model="llama3-70b-8192",
    )
    return completion.choices[0].message.content


In [52]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure of the floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kilogram

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this with 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944x10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944x10e25 Kilograms.

Now it's your turn:
""".strip()

In [53]:
# tools
def calculate(operation):
  return eval(operation)

def get_planet_mass(planet) -> float:
  match planet.lower():
    case "earth":
      return 5.972e24
    case "mars":
      return 6.39e23
    case "jupiter":
      return 1.898e27
    case "saturn":
      return 5.683e26
    case "uranus":
      return 8.681e25
    case "neptune":
      return 1.024e26
    case "mercury":
      return 3.285e23
    case "venus":
      return 4.867e24
    case _:
      return 0.0

In [54]:
neil_tyson = Agent(client, system_prompt)

In [55]:
result = neil_tyson("What is the mass of Earth times 5?")
print(result)

Thought: I need to find the mass of Earth


In [56]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure of the floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kilogram\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth \nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this with 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944x10e25\n\nIf you have 

In [57]:
observation = get_planet_mass("Earth")
print(observation)

5.972e+24


In [58]:
next_prompt = f"Observation : {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: I need to multiply this with 5


In [59]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure of the floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kilogram\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth \nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this with 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944x10e25\n\nIf you have 

In [60]:
observation = calculate("5.972e24 * 5")
print(observation)

2.9860000000000004e+25


In [61]:
next_prompt = f"Observation : {observation}"
result = neil_tyson(next_prompt)
print(result)

Answer: The mass of Earth times 5 is 2.9860000000000004e+25 Kilograms.


In [62]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure of the floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kilogram\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth \nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this with 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944x10e25\n\nIf you have 

In [66]:
import re

def agent_loop(max_iterations, system, query):
  agent = Agent(client, system_prompt)
  tools = ['calculate', 'get_planet_mass']
  next_prompt = query
  i = 0
  while i < max_iterations:
    i += 1
    result = agent(next_prompt)
    print(result)

    if "PAUSE" in result and "Action" in result:
      action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
      chosen_tool = action[0][0]
      arg = action[0][1]

      if chosen_tool in tools:
        result_tool = eval(f"{chosen_tool}('{arg}')")
        next_prompt = f"Observation: {result_tool}"

      else:
        next_prompt = "Observation: Tool not found"

      print(next_prompt)
      continue

    if "Answer" in result:
      break

In [67]:
agent_loop(max_iterations=10, system = system_prompt, query= "What is the mass of earth plus the mass of mercury and all of it times 5?")

Thought: I need to find the mass of Earth and Mercury, and then multiply it by 5.
Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: I have the mass of Earth, now I need to find the mass of Mercury.
Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: I have the masses of both planets, now I need to add them together.
Action: calculate: 5.972e+24 + 3.285e+23
PAUSE
Observation: 6.300500000000001e+24
Thought: Now I need to multiply the sum by 5.
Action: calculate: 6.300500000000001e+24 * 5
PAUSE
Observation: 3.1502500000000004e+25
Thought: I have the result, now I can output the answer.

Answer: The mass of Earth plus the mass of Mercury and all of it times 5 is 3.1502500000000004e+25 Kilograms.
